In [1]:
# USER OPTIONS
# define tokenizer name - should be one among the keys in the cell below
tokenizer_name = 'ChordSymbolTokenizer' # or any other name from the keys in tokenizers dictionary
# tokenizer_name = 'RootTypeTokenizer'
# tokenizer_name = 'PitchClassTokenizer'
# folder to xmls
val_dir = '/media/maindisk/maximos/data/hooktheory_test'
# val_dir = '/media/maindisk/maximos/data/gjt_melodies/Library_melodies'
# val_dir = '/media/datadisk/datasets/gjt_melodies/Library_melodies'
# val_dir = '/media/maximos/9C33-6BBD/data/gjt_melodies/Library_melodies'

# define batch size depending on GPU availability / status
batchsize = 16
# select device name - could be 'cpu', 'cuda', 'coda:0', 'cuda:1'...
device_name = 'cuda'

In [2]:
import pickle
with open('data/idioms_mir_quick_reference.pickle', 'rb') as f:
    idioms = pickle.load(f)
with open('data/idioms_mir_full_info.pickle', 'rb') as f:
    info_idioms = pickle.load(f)

In [3]:
bc_major = idioms['BachChorales_[0 2 4 5 7 9 11]']
bc_minor = idioms['BachChorales_[0 2 3 5 7 8 10]']
jazz_maj = idioms['Jazz_[0 2 4 5 7 9 11]']
jazz_min = idioms['Jazz_[0 2 3 5 7 9 10]']
organum = idioms['organum_[0 2 3 5 7 8 10]']

In [4]:
for c in info_idioms['organum']['[0 2 3 5 7 8 10]']['chords'].values():
    print(c['match_symbol'])

A:5
C:5
D:5
E:5
D#:aug
A:1
G:1
F:5
D:1
C:1
D:1
E:1
G:5
B:1
D:min
B:5
C:maj


In [5]:
from data_utils import SeparatedMelHarmMarkovDataset
import os
import numpy as np
from harmony_tokenizers_m21 import ChordSymbolTokenizer, RootTypeTokenizer, \
    PitchClassTokenizer, RootPCTokenizer, GCTRootPCTokenizer, \
    GCTSymbolTokenizer, GCTRootTypeTokenizer, MelodyPitchTokenizer, \
    MergedMelHarmTokenizer
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import BartForConditionalGeneration, BartConfig, DataCollatorForSeq2Seq
from tqdm import tqdm
from models import TransGraphVAE
import csv
from transformers import LogitsProcessor, StoppingCriteria, StoppingCriteriaList

/home/maximos/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
tokenizers = {
    'ChordSymbolTokenizer': ChordSymbolTokenizer,
    'RootTypeTokenizer': RootTypeTokenizer,
    'PitchClassTokenizer': PitchClassTokenizer,
    'RootPCTokenizer': RootPCTokenizer,
    'GCTRootPCTokenizer': GCTRootPCTokenizer,
    'GCTSymbolTokenizer': GCTSymbolTokenizer,
    'GCTRootTypeTokenizer': GCTRootTypeTokenizer
}

In [7]:
melody_tokenizer = MelodyPitchTokenizer.from_pretrained('saved_tokenizers/MelodyPitchTokenizer')
harmony_tokenizer = tokenizers[tokenizer_name].from_pretrained('saved_tokenizers/' + tokenizer_name)

tokenizer = MergedMelHarmTokenizer(melody_tokenizer, harmony_tokenizer)

bart_path = 'saved_models/bart/' + tokenizer_name + '/' + tokenizer_name + '.pt'

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
bart_config = BartConfig(
    vocab_size=len(tokenizer.vocab),
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    decoder_start_token_id=tokenizer.bos_token_id,
    forced_eos_token_id=tokenizer.eos_token_id,
    max_position_embeddings=512,
    encoder_layers=8,
    encoder_attention_heads=8,
    encoder_ffn_dim=512,
    decoder_layers=8,
    decoder_attention_heads=8,
    decoder_ffn_dim=512,
    d_model=512,
    encoder_layerdrop=0.3,
    decoder_layerdrop=0.3,
    dropout=0.3
)

bart = BartForConditionalGeneration(bart_config)

if torch.cuda.is_available():
    checkpoint = torch.load(bart_path, weights_only=True)
else:
    checkpoint = torch.load(bart_path, map_location="cpu", weights_only=True)
bart.load_state_dict(checkpoint)

/home/maximos/.local/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


<All keys matched successfully>

In [10]:
enc1 = bart.model.encoder
enc2 = bart.get_encoder()
dec1 = bart.model.decoder
dec2 = bart.get_decoder()

In [11]:
dec1 == dec2

True

In [12]:
test_dir = '/mnt/ssd2/maximos/data/hooktheory_test'
test_dataset = SeparatedMelHarmMarkovDataset(test_dir, tokenizer, max_length=512, num_bars=64)

# Data collator for BART
def create_data_collator(tokenizer, model):
    return DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)
# end create_data_collator

In [13]:
collator = create_data_collator(tokenizer, model=bart)
valloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=collator)

In [14]:
model_path = 'saved_models/bart_cvae/' + tokenizer_name + '/' + tokenizer_name + '.pt'

config = {
    'hidden_dim_LSTM': 1024,
    'hidden_dim_GNN': 1024,
    'latent_dim': 1024,
    'condition_dim': 1024,
    'use_attention': False
}

model = TransGraphVAE(transformer=bart, device=device, tokenizer=tokenizer, **config)

if torch.cuda.is_available():
    checkpoint = torch.load(model_path, weights_only=True)
else:
    checkpoint = torch.load(model_path, map_location="cpu", weights_only=True)
model.load_state_dict(checkpoint)

model.to(device)
model.eval()

TransGraphVAE(
  (transformer): BartForConditionalGeneration(
    (model): BartModel(
      (shared): BartScaledWordEmbedding(545, 512, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): BartScaledWordEmbedding(545, 512, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(514, 512)
        (layers): ModuleList(
          (0-7): 8 x BartEncoderLayer(
            (self_attn): BartSdpaAttention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=512, out_features=512, bias=True)
            (fc2): Linear(in_feat

In [15]:
batch = next(iter(valloader))

/home/maximos/.local/lib/python3.11/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
/home/maximos/.local/lib/python3.11/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


In [16]:
with torch.no_grad():
    input_ids = batch['input_ids'].to(device)
    # transitions = batch['transitions'].to(device)
    # transitions = torch.FloatTensor(bc_major).reshape(1,bc_major.shape[0], bc_major.shape[1]).to(device)
    # transitions = torch.FloatTensor(bc_minor).reshape(1,bc_minor.shape[0], bc_minor.shape[1]).to(device)
    transitions = torch.FloatTensor(organum).reshape(1,organum.shape[0], organum.shape[1]).to(device)
    # transitions = torch.FloatTensor(jazz_maj).reshape(1,jazz_maj.shape[0], jazz_maj.shape[1]).to(device)
    # transitions = torch.FloatTensor(jazz_min).reshape(1,jazz_min.shape[0], jazz_min.shape[1]).to(device)
    # attention_mask = batch['attention_mask'].to(device)
    attention_mask = None
    output_tokens = []
    output_recon_tokens = []
    num_bars = (input_ids == tokenizer.vocab['<bar>']).sum(dim=1).reshape(input_ids.shape[0],-1)
    print(input_ids)
    print(transitions)
    print(num_bars.shape)
    outputs = model(input_ids, transitions, encoder_attention=attention_mask, generate_max_tokens=500, num_bars=num_bars, temperature=1.0)
    for i in outputs['generated_ids'][0]:
        output_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
    for i in outputs['generated_recon_ids'][0]:
        output_recon_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )

tensor([[  2,   6, 180,  95,  55, 103,  62, 107,  60, 115,  58, 123,  57,   6,
          95,  57,  99,  58, 103,  57, 107,  55, 111,  55, 115,  53, 119,   4,
         123,  58,   6,  95,  58,  99,  57, 103,  53, 107,  55, 115,  53, 119,
          51, 123,  50,   6,  95,  50,  99,  51, 103,  53, 107,  60, 115,  58,
         119,   4,   6,  95,  55, 103,  62, 107,  60, 115,  58, 123,  63,   6,
          95,  63,  99,  62, 103,  60, 107,  58, 111,  57, 115,  58, 119,  60,
         123,  62,   6,  95,  62,   6,  95,   4, 103,  62, 111,  60, 119,  58,
           6,  95,  55, 103,  62, 107,  60, 115,  58, 123,  57,   6,  95,  57,
          99,  58, 103,  57, 107,  55, 111,  55, 115,  53, 119,   4, 123,  58,
           6,  95,  58,  99,  57, 103,  53, 107,  55, 115,  53, 119,  51, 123,
          50,   6,  95,  50,  99,  51, 103,  53, 107,  60, 115,  58, 119,   4,
           6,  95,  55, 103,  62, 107,  60, 115,  58, 123,  63,   6,  95,  63,
          99,  62, 103,  60, 107,  58, 111,  57, 115

In [17]:
print(transitions.shape)

torch.Size([1, 348, 348])


In [18]:
print(output_tokens)

['<s>', '<h>', '<bar>', 'position_0x00', 'A:min', 'position_3x00', 'G:7', '<bar>', 'position_0x00', 'G:7', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'F:maj7', 'position_2x00', 'C:maj', 'position_3x00', 'D:min', '<bar>', 'position_0x00', 'E:7', 'position_1x50', 'A:min', 'position_3x50', 'C:maj', '<bar>', 'position_0x00', 'C:maj', 'position_1x50', 'G:maj', 'position_1x50', 'F:maj', '<bar>', 'position_0x00', 'C:maj', 'position_1x50', 'F:maj', '<bar>', 'position_0x00', 'G:sus4', '<bar>', 'position_0x00', 'G:maj', 'position_1x50', 'E:maj', '<bar>', 'position_0x00', 'C:maj', 'position_1x50', 'G:maj', '<bar>', 'position_0x00', 'A:min', 'position_1x50', 'E:maj', '<bar>', 'position_0x00', 'C:maj', 'position_1x50', 'F:maj', '<bar>', 'position_0x00', 'C:maj', 'position_1x50', 'G:maj', 'position_3x50', 'A:min', '<bar>', 'position_0x00', 'A:min', 'position_1x50', 'C:maj', '<bar>', 'position_0x00', 'F:maj', 'position_1x50', 'E:min', '<bar>', 'position_0x00', 'D:min', 'position_1x5

In [19]:
print(output_recon_tokens)

['<s>', '<h>', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'C:maj', 'position_2x00', 'C:maj', 'position_3x00', 'C:maj', 'position_3x50', 'C:maj', 'G#:hdim7', '<bar>', 'position_0x00', 'F#:sus4', 'position_1x50', 'E:min', 'position_2x00', 'E:min', 'position_3x50', 'F#:sus4', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'E:min7', 'position_2x00', 'E:min', 'position_3x00', 'G:sus2', 'position_3x50', 'G:maj', 'position_3x75', 'G:7', 'position_3x75', 'F:maj', '<bar>', 'position

In [20]:
input_ids = batch['input_ids'].to(device)
print(input_ids)
input_tokens = []
for i in input_ids[0]:
    input_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
print(input_tokens)

tensor([[  2,   6, 180,  95,  55, 103,  62, 107,  60, 115,  58, 123,  57,   6,
          95,  57,  99,  58, 103,  57, 107,  55, 111,  55, 115,  53, 119,   4,
         123,  58,   6,  95,  58,  99,  57, 103,  53, 107,  55, 115,  53, 119,
          51, 123,  50,   6,  95,  50,  99,  51, 103,  53, 107,  60, 115,  58,
         119,   4,   6,  95,  55, 103,  62, 107,  60, 115,  58, 123,  63,   6,
          95,  63,  99,  62, 103,  60, 107,  58, 111,  57, 115,  58, 119,  60,
         123,  62,   6,  95,  62,   6,  95,   4, 103,  62, 111,  60, 119,  58,
           6,  95,  55, 103,  62, 107,  60, 115,  58, 123,  57,   6,  95,  57,
          99,  58, 103,  57, 107,  55, 111,  55, 115,  53, 119,   4, 123,  58,
           6,  95,  58,  99,  57, 103,  53, 107,  55, 115,  53, 119,  51, 123,
          50,   6,  95,  50,  99,  51, 103,  53, 107,  60, 115,  58, 119,   4,
           6,  95,  55, 103,  62, 107,  60, 115,  58, 123,  63,   6,  95,  63,
          99,  62, 103,  60, 107,  58, 111,  57, 115

In [21]:
class ExactTokenCountLogitsProcessor(LogitsProcessor):
    def __init__(self, token_id, max_count):
        self.token_id = token_id
        self.max_count = max_count

    def __call__(self, input_ids, scores):
        token_count = (input_ids == self.token_id).sum().item()
        if token_count >= self.max_count:
            scores[:, self.token_id] = -float("inf")  # Mask the token
        return scores

class ExactTokenCountStoppingCriteria(StoppingCriteria):
    def __init__(self, token_id, max_count):
        self.token_id = token_id
        self.max_count = max_count

    def __call__(self, input_ids, scores, **kwargs):
        token_count = (input_ids == self.token_id).sum().item()
        return token_count >= self.max_count  # Stop when the count is reached


In [22]:
# Define the token ID and the exact count you want
token_id = 200
exact_count = 10

bart_outputs = bart.generate(
    input_ids=input_ids,
    # attention_mask=batch['attention_mask'][bi],
    bos_token_id=tokenizer.vocab[tokenizer.harmony_tokenizer.start_harmony_token],
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=500,
    do_sample=True,
    temperature=1.0,
    # logits_processor=[ExactTokenCountLogitsProcessor(token_id, exact_count)],
    # stopping_criteria=StoppingCriteriaList([ExactTokenCountStoppingCriteria(token_id, exact_count)])
)

In [23]:
print(bart_outputs)
bart_only_outputs = []
for i in bart_outputs[0]:
    bart_only_outputs.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
print(bart_only_outputs)

tensor([[  2, 196,   6,  95, 459, 107, 431, 107, 197, 107, 256, 115, 314,   6,
          95, 459,   6,  95, 197, 107, 400,   6,  95, 197, 107, 349,   6,  95,
         342, 107, 197,   6,  95, 400, 107, 197,   6,  95, 197, 107, 400,   6,
          95, 459, 107, 197,   6,  95, 342,   6,  95, 197, 107, 197,   6,  95,
         459, 107, 314,   6,  95, 400,   6,  95, 256,   6,  95, 314, 111, 197,
           6,  95, 342,   6,  95, 400, 111, 314,   6,  95, 314,   6,  95, 342,
         111, 197,   6,  95, 256,   6,  95, 256, 111, 313,   6,  95, 342,   6,
          95, 400,   6,  95, 197,   6,  95, 256, 111, 313,   6,  95, 404,   6,
          95, 404,   6,  95, 404,   6,  95, 404, 111, 400,   3]],
       device='cuda:0')
['<s>', '<h>', '<bar>', 'position_0x00', 'A:min', 'position_1x50', 'G#:aug', 'position_1x50', 'C:maj', 'position_1x50', 'D:min', 'position_2x50', 'E:min', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'C:maj', 'position_1x50', 'G:maj', '<bar>', 'position_0x00', '

In [24]:
print(bart_only_outputs)
print(output_tokens)
print(output_recon_tokens)

['<s>', '<h>', '<bar>', 'position_0x00', 'A:min', 'position_1x50', 'G#:aug', 'position_1x50', 'C:maj', 'position_1x50', 'D:min', 'position_2x50', 'E:min', '<bar>', 'position_0x00', 'A:min', '<bar>', 'position_0x00', 'C:maj', 'position_1x50', 'G:maj', '<bar>', 'position_0x00', 'C:maj', 'position_1x50', 'F:maj7', '<bar>', 'position_0x00', 'F:maj', 'position_1x50', 'C:maj', '<bar>', 'position_0x00', 'G:maj', 'position_1x50', 'C:maj', '<bar>', 'position_0x00', 'C:maj', 'position_1x50', 'G:maj', '<bar>', 'position_0x00', 'A:min', 'position_1x50', 'C:maj', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'C:maj', 'position_1x50', 'C:maj', '<bar>', 'position_0x00', 'A:min', 'position_1x50', 'E:min', '<bar>', 'position_0x00', 'G:maj', '<bar>', 'position_0x00', 'D:min', '<bar>', 'position_0x00', 'E:min', 'position_2x00', 'C:maj', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'G:maj', 'position_2x00', 'E:min', '<bar>', 'position_0x00', 'E:min', '<bar>', 'position_0x00'

In [25]:
tokenizer.decode( input_tokens + bart_only_outputs[1:], output_format='file', output_path='examples/bart.mxl' )
tokenizer.decode( input_tokens + output_tokens[1:], output_format='file', output_path='examples/encdec.mxl' )
tokenizer.decode( input_tokens + output_recon_tokens[1:], output_format='file', output_path='examples/recon.mxl' )

Saved as examples/bart.mxl
Saved as examples/encdec.mxl
Saved as examples/recon.mxl


In [26]:
print( (outputs['generated_markov']-transitions.to('cpu')).pow(2).sum().sqrt() )
print( (outputs['recon_markov']-transitions.to('cpu')).pow(2).sum().sqrt() )

tensor(37.9221, dtype=torch.float64)
tensor(31.6931, dtype=torch.float64)
